라이브러리

In [9]:
import numpy as np
from collections import OrderedDict


1. 신경망 학습 전체 과정 요약
    - 신경망의 학습은 4단계로 구성된다.




| 단계 | 설명 |
|:----|:----|
| 1단계 | 미니배치: 훈련 데이터 일부를 무작위로 선택 |
| 2단계 | 기울기 산출: 미니배치에 대해 손실 함수의 기울기 계산 |
| 3단계 | 매개변수 갱신: 기울기 방향으로 가중치/편향 업데이트 |
| 4단계 | 반복: 1~3단계를 반복 수행 |


2. TwoLayerNet 클래스 구현
    - 2층 신경망 (입력층 → 은닉층(ReLU) → 출력층(Softmax))을 모듈화하여 구성한다.



- 2.1 클래스 구조

In [18]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
    

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None  # softmax 출력
        self.t = None  # 정답 레이블

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx



def numerical_gradient(f, x):
    h = 1e-4  # 아주 작은 값
    grad = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]

        x[idx] = float(tmp_val) + h
        fxh1 = f(x)  # f(x+h)

        x[idx] = tmp_val - h
        fxh2 = f(x)  # f(x-h)

        grad[idx] = (fxh1 - fxh2) / (2 * h)

        x[idx] = tmp_val  # 원래 값 복원
        it.iternext()

    return grad

In [19]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        # 마지막 계층
        self.lastLayer = SoftmaxWithLoss()
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

- 2.3 메서드 설명


| 메서드 | 설명 |
|:------|:-----|
| predict(x) | 입력 x에 대해 순전파 수행 |
| loss(x, t) | 손실 함수 값을 계산 |
| accuracy(x, t) | 정확도 계산 |
| numerical_gradient(x, t) | 수치 미분으로 기울기 계산 |
| gradient(x, t) | 오차역전파법으로 기울기 계산 |


3. 오차역전파법으로 구한 기울기 검증하기 (Gradient Check)
    - 오차역전파법으로 계산한 기울기와 수치 미분으로 계산한 기울기를 비교하여 정확성을 검증한다.


- 3.1 테스트 코드

In [20]:


if __name__ == '__main__':
    from tensorflow.keras.datasets import mnist

    # 데이터 읽기
    (x_train, t_train), (x_test, t_test) = mnist.load_data()
    
    # flatten (28x28 -> 784) & normalize (0~255 -> 0~1)
    x_train = x_train.reshape(x_train.shape[0], -1).astype(np.float32) / 255.0
    x_test = x_test.reshape(x_test.shape[0], -1).astype(np.float32) / 255.0

    # one-hot encoding
    def to_one_hot(t, num_classes=10):
        one_hot = np.zeros((t.size, num_classes))
        one_hot[np.arange(t.size), t] = 1
        return one_hot

    t_train = to_one_hot(t_train)
    t_test = to_one_hot(t_test)

    # 네트워크 생성
    network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

    x_batch = x_train[:3]
    t_batch = t_train[:3]

    grad_numerical = network.numerical_gradient(x_batch, t_batch)
    grad_backprop = network.gradient(x_batch, t_batch)

    # 두 방법으로 구한 기울기 비교
    for key in grad_numerical.keys():
        diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
        print(key + ":" + str(diff))


W1:0.0008926971166034764
b1:0.004650029974384556
W2:0.012073497734149861
b2:0.3070636193375435


| 항목 | 설명 |
|:----|:-----|
| TwoLayerNet | 2층 신경망 클래스로, 레이어 순서대로 forward/backward 처리 |
| 오차역전파법 | 기울기를 빠르고 정확하게 계산하는 방법 |
| 수치 미분 | 검증용으로만 사용 (느림) |
| Gradient Check | 오차역전파법과 수치 미분 결과를 비교해 구현 정확성 검증 |
